In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_screentime_raw = pd.concat([pd.read_csv(f) for f in glob.glob("../data/raw/screentime*.csv")]).drop_duplicates()
df_screentime_raw

,app,usage,start_time,end_time,created_at,tz,device_id,device_model
0,com.apple.finder,3,1752110682,1752110685,1.752111e+09,-18000,NaN,NaN
1,com.apple.Terminal,1,1752110681,1752110682,1.752111e+09,-18000,NaN,NaN
2,com.apple.finder,12,1752110669,1752110681,1.752111e+09,-18000,NaN,NaN
3,com.microsoft.VSCode,3,1752110666,1752110669,1.752111e+09,-18000,NaN,NaN
4,com.tableausoftware.tableaudesktop,86,1752110580,1752110666,1.752111e+09,-18000,NaN,NaN
...,...,...,...,...,...,...,...,...
3871,com.apple.MobileSMS,3,1720368014,1720368017,1.720368e+09,-18000,EB445137-D7F5-57EC-87DA-1DB460D64046,"iPhone13,2"
3872,com.apple.Health,85,1720366254,1720366339,1.720366e+09,-18000,EB445137-D7F5-57EC-87DA-1DB460D64046,"iPhone13,2"
3873,com.apple.findmy,10,1720366233,1720366243,1.720366e+09,-18000,EB445137-D7F5-57EC-87DA-1DB460D64046,"iPhone13,2"
3874,com.apple.mobiletimer,31,1720366200,1720366231,1.720366e+09,-18000,EB445137-D7F5-57EC-87DA-1DB460D64046,"iPhone13,2"


## Data Dictionary
Each row represents a screen time session.

- `app`: Name of the application used during the screen time session.
- `usage`: Duration of the screen time session in seconds.
- `start_time`: Timestamp of when the screen time session started.
- `end_time`: Timestamp of when the screen time session ended.
- `created_at`: Timestamp of when the record was created.
- `tz`: Timezone of the screen time session.
- `device_id`: Unique identifier for the device.
- `device_model`: Model of the device. Missing values imply the device was a macOS device.


In [3]:
df_screentime_raw.isna().sum()

app                 0
usage               0
start_time          0
end_time            0
created_at          0
tz                  0
device_id       68933
device_model    68933
dtype: int64

In [4]:
# sort by start_time
df_screentime_clean = df_screentime_raw.sort_values("start_time").reset_index(drop=True)

# convert timestamps to datetime
df_screentime_clean["start_time"] = df_screentime_clean["start_time"].apply(lambda x: datetime.fromtimestamp(x))
df_screentime_clean["end_time"] = df_screentime_clean["end_time"].apply(lambda x: datetime.fromtimestamp(x))
df_screentime_clean["created_at"] = df_screentime_clean["created_at"].apply(lambda x: datetime.fromtimestamp(x))

# create a new column for the date
df_screentime_clean["date"] = pd.to_datetime(df_screentime_clean["end_time"]).dt.date

# convert usage to hours
df_screentime_clean["usage"] = df_screentime_clean["usage"] / 60

# fill missing device_model with 'macbook_pro'
df_screentime_clean['device_model'] = df_screentime_clean['device_model'].fillna('macbook_pro')

# select relevant columns and rename
df_screentime_clean = df_screentime_clean[['date', 'start_time', 'end_time', 'usage', 'device_model', 'app']].rename(columns={"device_model": "device", "usage": "usage (min)", "app": "app_id"})

# merge app name and category data
app_categories = pd.read_csv("../data/raw/app_categories.csv")[['app_id', 'app_name', 'app_category']]
df_screentime_clean = df_screentime_clean.merge(app_categories, on='app_id', how='left')
df_screentime_clean['app_name'] = df_screentime_clean['app_name'].fillna(df_screentime_clean['app_id'])
df_screentime_clean['app_category'] = df_screentime_clean['app_category'].fillna('other')

df_screentime_clean


,date,start_time,end_time,usage (min),device,app_id,app_name,app_category
0,2024-01-22,2024-01-22 10:23:59,2024-01-22 10:30:12,6.216667,macbook_pro,com.google.Chrome,Chrome,browser
1,2024-01-22,2024-01-22 10:30:12,2024-01-22 10:30:15,0.050000,macbook_pro,com.google.Chrome,Chrome,browser
2,2024-01-22,2024-01-22 10:53:10,2024-01-22 11:02:12,9.033333,macbook_pro,com.google.Chrome,Chrome,browser
3,2024-01-22,2024-01-22 11:02:12,2024-01-22 11:02:35,0.383333,macbook_pro,com.microsoft.Outlook,Outlook,productivity
4,2024-01-22,2024-01-22 11:02:35,2024-01-22 11:35:52,33.283333,macbook_pro,com.google.Chrome,Chrome,browser
...,...,...,...,...,...,...,...,...
101208,2025-07-09,2025-07-09 20:23:00,2025-07-09 20:24:26,1.433333,macbook_pro,com.tableausoftware.tableaudesktop,Tableau,productivity
101209,2025-07-09,2025-07-09 20:24:26,2025-07-09 20:24:29,0.050000,macbook_pro,com.microsoft.VSCode,VSCode,productivity
101210,2025-07-09,2025-07-09 20:24:29,2025-07-09 20:24:41,0.200000,macbook_pro,com.apple.finder,Finder,productivity
101211,2025-07-09,2025-07-09 20:24:41,2025-07-09 20:24:42,0.016667,macbook_pro,com.apple.Terminal,Terminal,productivity


In [5]:
# group by date, device, and app_category, summing usage
df_screentime_cat_agg = df_screentime_clean.groupby(['date', 'device', 'app_category']).agg({'usage (min)': 'sum'}).reset_index()

# ensure dataset contains entries for categories with zero usage in a category on a given day
df_screentime_cat_agg = (
    df_screentime_cat_agg
    .pivot_table(index=['date', 'device'], columns='app_category', values='usage (min)', fill_value=0)
    .reset_index()
    .melt(id_vars=['date', 'device'], var_name='app_category', value_name='usage (min)')
)
# separate mac and ios usage
df_screentime_cat_agg = df_screentime_cat_agg.pivot(columns='device', values='usage (min)', index=['date', 'app_category']).fillna(0).reset_index().rename(columns={"macbook_pro": "usage_mac (min)", "iPhone13,2": "usage_phone (min)"})
df_screentime_cat_agg.columns.name = None

# detect days with zero phone usage and remove them (inaccurate data)
zero_phone_data_mask = df_screentime_cat_agg.groupby('date')['usage_phone (min)'].sum()
zero_phone_data_mask = zero_phone_data_mask[zero_phone_data_mask == 0].index
df_screentime_cat_agg = df_screentime_cat_agg[~df_screentime_cat_agg['date'].isin(zero_phone_data_mask)]

df_screentime_cat_agg.to_csv("../data/processed/screentime_by_category.csv", index=False)
df_screentime_cat_agg

,date,app_category,usage_phone (min),usage_mac (min)
19,2024-01-23,browser,10.816667,124.8
20,2024-01-23,communication,5.966667,3.5
21,2024-01-23,dasher,0.000000,0.0
22,2024-01-23,data entry,12.383333,0.0
23,2024-01-23,finance,1.183333,0.0
...,...,...,...,...
8317,2025-07-07,shopping,0.000000,0.0
8318,2025-07-07,social,1.666667,0.0
8319,2025-07-07,streaming,0.000000,0.0
8320,2025-07-07,utilities,0.000000,0.0
